# Prediction of Global Food Security Index (GFSI) Using RASFF Portal Data (Learning from Networks 2023-2024)

Alberto Battiston (2086522, alberto.battiston.1@studenti.unipd.it) <br>
Daniele Moschetta (2087640, daniele.moschetta@studenti.unipd.it) <br>
Francesco Visentin (2083245, francesco.visentin.6@studenti.unipd.it)

To code has been developed and tested using Google Colab.

Execution over a local Python 3.10+ enviroment is possible but some extra dependecies may be required. (Eg. PyTorch, scikit-learn, NetworkX...)<br>
To run locally and install the needed packages uncomment the code below.

In [ ]:
# Uncomment if running on a local Python 3.10+ enviroment
#!pip install -U numpy
#!pip install -U pandas
#!pip install -U torch
#!pip install -U networkx
#!pip install -U scikit-learn
#!pip install -U matplotlib

## 0 - Setup the environment

Install Basemap toolkit, needed for plotting the graph, and PyTorchGeometric

In [ ]:
!pip install -U basemap
!pip install -U torch_geometric

Import libraries

In [ ]:
# General
import pandas as pd
import numpy as np
from collections import defaultdict
import copy

# Graphs
import networkx as nx

# Plotting
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap as Basemap

# Machine learning: linear model approach
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Machine learning: MLP and GNN
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader

## 1 - Read the datasets and extract the needed information

Set the GithHub repository as the base path to access the datasets directly from Google Colab. If running locally change to the local repo base path

In [ ]:
base_path = "https://raw.githubusercontent.com/FrancescoVisentin/RASFF_Analysis_LFN/main"

Read the RASFF dataset

In [ ]:
RASFF_data = pd.read_csv(base_path+'/data/RASFF_2012-2022.csv', on_bad_lines = 'skip')
RASFF_data.head()

Read the GFSI scores

In [ ]:
GFSI_scores = pd.read_csv(base_path+'/data/GFSI_2012-2022.csv')
GFSI_scores.head()

Read the geographical information of countries (latitude and longitude)

In [ ]:
countries_pos = pd.read_csv(base_path+'/data/countries_coordinates.csv', on_bad_lines = 'skip')
countries_pos.head()

Define the basemap object that is used to plot the nodes on a geographical map

In [ ]:
basemap = Basemap(projection='merc', llcrnrlat=-80, urcrnrlat=80, llcrnrlon=-180, urcrnrlon=180, lat_ts=0, resolution='l', suppress_ticks=True)

Define a function that takes in input the dataset and extracts the information needed to build the graph. In particular, for each country we keep track of the number of times it is appears in the dataset, considering its role and the risk associated to each notification. Geographical information needed for visualization are also extracted.


In [ ]:
def get_data(RASFF_data, countriesInfo, basemap):
    coordinates_dict = {}
    notifications_data = []
    risk_dict = defaultdict(lambda : {"undecided":0, "serious":0, "potentially serious":0, "potential risk":0, "not serious":0, "no risk":1})
    frequency_dict = defaultdict(lambda : {"origin":0, "operator":0, "destination":0})

    # Gets info required for the data visualization
    for _, row in countries_pos.iterrows():
        coordinates_dict[row["country"]] = basemap(row['longitude'], row['latitude'])

    # Gets a record of each notification
    for _, row in RASFF_data.iterrows():
        origins = [orig for orig in str(row['origins']).split(",") if orig != "nan"]
        operators = [oper for oper in str(row['operators']).split(",") if oper != "nan"]
        destinations = [dist for dist in str(row['destinations']).split(",") if dist != "nan"]
        risk = str(row["risk"])

        # Save the notification as a tuple
        notifications_data.append((origins, operators, destinations))

        # Extract the needed info from each notification
        for origin in origins:
            frequency_dict[origin]["origin"] += 1
            for operator in operators:
                frequency_dict[operator]["operator"] += 1
                risk_dict[origin][risk] += 1

        for operator in operators:
            for destination in destinations:
                frequency_dict[destination]["destination"] += 1
                risk_dict[operator][risk] += 1

    return notifications_data, coordinates_dict, frequency_dict, risk_dict

Define a function that, given the list of notifications and the information extracted above builds the graph.<br>
The result is a directed weighted graph.

In [ ]:
def build_graph(data, risk_dict, weight_fun):

    edges = defaultdict(lambda : .0)
    for notification in data:
        origins, operators, destinations = notification

        # Connects origins to operators (many-to-many)
        for origin in origins:
            for operator in operators:
                edges[(origin, operator)] += weight_fun(origin, operators)

        # Connects operators to destinations (many-to-many)
        for operator in operators:
            for destination in destinations:
                edges[(operator, destination)] += weight_fun(operator, destinations)

    # Build the graph
    graph = nx.DiGraph()
    for e, w in edges.items():
        graph.add_edge(e[0], e[1], weight=w)

    return graph

Define function for plotting the graph. The plot will be different depending on the metrics that we specify, in order to highlight the results. If no metric is speicified, the graph is simply plotted as it is.

In [ ]:
PLOT_NODE_SIZE = 50
PLOT_LINE_WIDTH = 0.5

def plot_graph(basemap: Basemap, graph: nx.DiGraph, coordinates_dict: dict,  metrics_dict: dict = None,
              node_color: str = 'red', edge_color: str = 'black', show_edges = False, show_labels = False):

    plt.figure(figsize = (10,9))

    if metrics_dict:
        color_factor = 250/max(metrics_dict.values())
        colors = ['#%02x%02x%02x' % (255, 255-int(metrics_dict[node]*color_factor), 0) for node in graph.nodes()]

        size_factor = PLOT_NODE_SIZE * 4 / max(metrics_dict.values())
        nx.draw_networkx_nodes(graph, coordinates_dict, node_color=colors,  edgecolors='black', linewidths=PLOT_LINE_WIDTH,
                               node_size = [metrics_dict[node] * size_factor for node in graph.nodes()])
    else:
        nx.draw_networkx_nodes(graph, coordinates_dict, node_color = node_color,  edgecolors='black', linewidths=PLOT_LINE_WIDTH,
                               node_size=PLOT_NODE_SIZE)

    if show_edges:
        nx.draw_networkx_edges(graph, coordinates_dict, arrows = True, edge_color=edge_color, width = PLOT_LINE_WIDTH)

    if show_labels:
        nx.draw_networkx_labels(graph, coordinates_dict, font_size=5, font_color='black')

    basemap.drawcountries(linewidth = PLOT_LINE_WIDTH)
    basemap.drawcoastlines(linewidth = PLOT_LINE_WIDTH)

    plt.tight_layout()
    plt.show()

## 2 - Build the graphs and compute the needed metrics

Choose the weight function

In [ ]:
WEIGHT_FUN_ID = 'W3'

# Set the weight function
match WEIGHT_FUN_ID:
    case 'W2':
        weight_fun = lambda country, out_countries: risk_dict[country]["serious"] / sum(risk_dict[country].values())
    case 'W3':
        weight_fun = lambda country, out_countries: len(out_countries) * risk_dict[country]["serious"] / (sum(risk_dict[country].values()) - risk_dict[country]["undecided"])
    case _:
        weight_fun = lambda country, out_countries: 1

Now, we are ready to call the previously defined functions and actually build the graphs used for our analysis

In [ ]:
graphs = []
coordinates_dicts = []
frequency_dicts = []
risk_dicts = []

# Convert dates to datetime
RASFF_data['date'] = pd.to_datetime(RASFF_data['date'])

first_year = RASFF_data['date'].dt.year.min()
last_year = RASFF_data['date'].dt.year.max()

for year in range(first_year, last_year + 1):
    # Filter the records by year
    cur_data = RASFF_data[RASFF_data['date'].dt.year == year]

    # Extract info from the dataset and returns the data structures used to build graph
    notifications_data, coordinates_dict, frequency_dict, risk_dict = get_data(cur_data, countries_pos, basemap)

    # Build graph
    graph = build_graph(notifications_data, risk_dict, weight_fun)

    # Add year data to data lists
    graphs.append(graph)
    coordinates_dicts.append(coordinates_dict)
    frequency_dicts.append(frequency_dict)
    risk_dicts.append(risk_dict)

Plot the graph nodes and edges

In [ ]:
# Plot the graph without any metric specified
plot_graph(basemap, graphs[0], coordinates_dicts[0], show_edges=True)

Define a function that is used for computing the main metrics of the graph:


* In-degree
* Out-degree
* Betwenness centrality
* Clustering Coefficient
* Origin score
* Risk score



In [ ]:
graphs_metrics = []

for i in range(len(graphs)):
    # Compute metrics
    in_deg = nx.in_degree_centrality(graphs[i])
    out_deg = nx.out_degree_centrality(graphs[i])
    bwc = nx.betweenness_centrality(graphs[i], normalized=True, weight="weight")
    cc = nx.closeness_centrality(graphs[i].reverse(), distance="weight")

    # Combine metrics into a single dict
    metrics_dict = {}
    for country in in_deg.keys():
        origin_score = frequency_dicts[i][country]["origin"] / sum(frequency_dicts[i][country].values())
        risk_score = (risk_dicts[i][country]["serious"]) / sum(risk_dicts[i][country].values())
        metrics_dict[country] = [origin_score, risk_score, in_deg[country], out_deg[country], bwc[country], cc[country]]

    # Add year metrics to metrics list
    graphs_metrics.append(metrics_dict)

Now, we plot the graph based on the metric that we want to visualize. For the sake of brevity, only two of them are riported but it's sufficient to specify the desired metric to visualize it. The higher the metric value, the more red is the color of the node.

In [ ]:
# Plot the In-Degree metric
in_deg = nx.in_degree_centrality(graphs[0])
plot_graph(basemap, graphs[0], coordinates_dicts[0], metrics_dict=in_deg)

In [ ]:
# Plot the Betwennes centrality metric
bwc = nx.betweenness_centrality(graphs[0], normalized=True, weight="weight")
plot_graph(basemap, graphs[0], coordinates_dicts[0], metrics_dict=bwc)

## 3 - Linear Model approach

Compute the feature vectors

In [ ]:
countries_features = []
last_year_index = -1

# For each year
for i in range(len(graphs_metrics)):
    cur_year = first_year + i

    # Save index of the first feature of the last year
    if cur_year == last_year:
        last_year_index = len(countries_features)

    # For each country
    for country in graphs_metrics[i].keys():
        # If there is a correspondence in the scores dataset
        if str(cur_year) in GFSI_scores.columns and (GFSI_scores['country'] == country).any():
            # Append the features to the feature list
            country_score = GFSI_scores.loc[GFSI_scores['country'] == country, str(cur_year)].values[0]
            country_features = []
            country_features.extend(graphs_metrics[i][country])
            country_features.append(country_score)
            countries_features.append(country_features)

In [ ]:
countries_features = pd.DataFrame(countries_features)
countries_features.head()

Normalize and then split train and test

In [ ]:
# Extract X and y from the features
X = countries_features.iloc[:, :-1]
y = countries_features.iloc[:, -1:]

# Normalize feature vectors
scaler = MinMaxScaler()
X_normalized = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Split in train and test sets
X_train = X_normalized[:last_year_index]
X_test = X_normalized[last_year_index:]
y_train = y[:last_year_index]
y_test = y[last_year_index:]

Fit linear regression model on train

In [ ]:
# Create linear regression model
model = LinearRegression()

# Fit model on the train set
model.fit(X_train, y_train)

Predict labels on test

In [ ]:
# Predict scores for test set
y_pred = model.predict(X_test)

# Compute the error
print("Linear model mean L1 loss: ", mean_absolute_error(y_test, y_pred))
print("Linear model mean MSE loss: ", mean_squared_error(y_test, y_pred))

Print predictions comparison

In [ ]:
countries = []
model_predictions = []
true_labels = []

# For each country
i = 0
for country in graphs_metrics[-1].keys():
    # If there is a correspondence in the scores dataset
    if str(cur_year) in GFSI_scores.columns and (GFSI_scores['country'] == country).any():
        countries.append(country)
        model_predictions.append(y_pred[i][0])
        true_labels.append(y_test.iloc[i, 0])
        i += 1

predictions_dict = {'Country': countries, 'Model prediction': model_predictions, 'GSFI score': true_labels}
predictions_df = pd.DataFrame(predictions_dict)
predictions_df = predictions_df.sort_values(by='Model prediction', ascending=False)
predictions_df = predictions_df.reset_index(drop=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.precision', 1,):
    display(predictions_df)

## 4 - MLP approach

Firstly we define a function to convert the graph to torch_geometric.data.Data and normalize the features

In [ ]:
def graph_to_torch_data(graph: nx.DiGraph, graph_index: int):

    year = first_year + graph_index

    # Compute node ids, features and labels
    node_ids = {}
    cur_id = 0
    node_features = []
    node_labels = []
    for country in graph.nodes():
        if str(year) in GFSI_scores.columns and (GFSI_scores['country'] == country).any():
            # Ids
            node_ids[country] = cur_id
            cur_id += 1
            # Features
            node_features.append(graphs_metrics[graph_index][country])
            # Label
            country_score = GFSI_scores.loc[GFSI_scores['country'] == country, str(year)].values[0]
            node_labels.append([country_score])

    # Compute edge indexes and attributes
    edge_index = []
    edge_attr = []
    for u,v,w in graph.edges(data=True):
        if u in node_ids and v in node_ids:
            edge_index.append([node_ids[u], node_ids[v]])
            edge_attr.append([w["weight"]])

    # Convert to tensors
    node_features = torch.tensor(node_features, dtype=torch.float)
    node_labels = torch.tensor(node_labels, dtype=torch.float)
    edge_index = torch.tensor(edge_index, dtype=torch.long)
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)

    # Normalization
    node_features = F.normalize(node_features)

    # Create torch_geometric.data.Data
    data = Data(x=node_features, y=node_labels, edge_index=edge_index.t().contiguous(), edge_attr=edge_attr)
    data.num_nodes = len(node_features)
    data.validate()

    return data, node_ids

Then two general functions to test our models and print the predicted labels

In [ ]:
def test_model(model: torch.nn.Module, data: Data):
    model.eval()
    with torch.no_grad():
        pred = model(data)
    return pred

In [ ]:
def print_predictions(pred: list, node_ids: dict):
  pred_scores = {}
  for i, score in enumerate(pred):
      country = [c for c in node_ids if node_ids[c] == i][0]
      pred_scores[country] = score[0]

  countries = []
  model_predictions = []
  true_labels = []
  for country, score in sorted(pred_scores.items(), key=lambda i : i[1], reverse=True):
      countries.append(country)
      model_predictions.append(score.item())
      true_labels.append(GFSI_scores.loc[GFSI_scores['country'] == country, str(last_year)].values[0])

  predictions_dict = {'Country': countries, 'Model prediction': model_predictions, 'GSFI score': true_labels}
  predictions_df = pd.DataFrame(predictions_dict)
  with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.precision', 1,):
    display(predictions_df)

Now we can define our MLP model and define a function to train it

In [ ]:
class MLP(torch.nn.Module):
    def __init__(self, input_features, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.lin1 = Linear(input_features, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1)

    def forward(self, data: Data):
        x = data.x

        x = self.lin1.forward(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin2.forward(x)
        x = 100 * F.sigmoid(x)

        return x

In [ ]:
def train_MLP(model, train_data: Data, val_data: Data, max_iter=1000, patience=20):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-4)
    criterion = torch.nn.MSELoss()

    min_val_loss = float('inf')
    best_model = copy.deepcopy(model)
    no_improv_count = 0

    for epoch in range(1, max_iter + 1):
        # Training phase
        model.train()
        optimizer.zero_grad()
        out = model(train_data)
        loss = criterion(out, train_data.y)
        loss.backward()
        optimizer.step()
        train_loss = loss.item()

        # Validation phase
        model.eval()
        with torch.no_grad():
            out = model(val_data)
            loss = criterion(out, val_data.y)
            val_loss = loss.item()

        print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

        # Early stopping check
        if val_loss < min_val_loss:
            min_val_loss = val_loss
            best_model = copy.deepcopy(model)
            no_improv_count = 0
        else:
            no_improv_count += 1

        if no_improv_count >= patience:
            print(f'Early stopping at epoch {epoch} due to no improvement in validation loss for {patience} epochs.')
            break

    return best_model

Split train, validation and test data

In [ ]:
data_list = []

# For each graph
for i in range(len(graphs)):
    data, node_ids = graph_to_torch_data(graphs[i], i)
    data_list.append(data)

# We need to "compact" the data from all the training graphs into a single input feature vectors
train_data_list = data_list[:-2]
train_data = Batch.from_data_list(train_data_list)

val_data = data_list[-2]
test_data = data_list[-1]

Create the MLP model and train it

In [ ]:
HIDDEN_CHANNLES = 20
MAX_EPOCH = 1000
EPOCH_PATIENCE = 60

model = MLP(data_list[0].num_node_features, HIDDEN_CHANNLES)

model = train_MLP(model, train_data, val_data, MAX_EPOCH, EPOCH_PATIENCE)

Predict labels on test data, compute the L1 and MSE loss

In [ ]:
# Get the model predictions
pred = test_model(model, test_data)

# Compute the associated loss
criterion = torch.nn.L1Loss()
loss = criterion(pred, test_data.y)
print(f"MLP model mean L1 loss: {loss.item():.3f}")

criterion = torch.nn.MSELoss()
loss = criterion(pred, test_data.y)
print(f"MLP model mean MSE loss: {loss.item():.3f}")

Print the predicted labels

In [ ]:
print_predictions(pred, node_ids)

## 5 - GNN Approach

Similarly to before we define the GNN model and a function to train it

In [ ]:
class GNN(torch.nn.Module):
    def __init__(self, input_features, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(input_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, 1)

    def forward(self, data: Data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

        x = self.conv1.forward(x, edge_index, edge_weight=edge_attr)
        x = F.relu(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv2.forward(x, edge_index, edge_weight=edge_attr)
        x = 100 * F.sigmoid(x)

        return x

In [ ]:
def train_GNN(model, train_loader: DataLoader, val_data: Data, max_iter=1000, patience=20):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-4)
    criterion = torch.nn.MSELoss()

    min_val_loss = float('inf')
    best_model = copy.deepcopy(model)
    stable_count = 0

    # Training phaseS
    for epoch in range(1, max_iter + 1):
        model.train()
        train_loss_sum = 0
        for data in train_loader:
            optimizer.zero_grad()
            out = model(data)
            loss = criterion(out, data.y)
            loss.backward()
            optimizer.step()
            train_loss_sum += loss.item()

        train_loss = train_loss_sum / len(train_loader.dataset)

        # Validation phase
        model.eval()
        with torch.no_grad():
            out = model(val_data)
            loss = criterion(out, val_data.y)
            val_loss = loss.item()

        print(f'Epoch: {epoch:03d}, Average train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

        # Early stopping check
        if val_loss < min_val_loss:
            min_val_loss = val_loss
            best_model = copy.deepcopy(model)
            no_improv_count = 0
        else:
            no_improv_count += 1

        if no_improv_count >= patience:
            print(f'Early stopping at epoch {epoch} due to no improvement in validation loss for {patience} epochs.')
            break

    return best_model

Combine train data into a DataLoader

In [ ]:
train_data_loader = DataLoader(train_data_list, batch_size = 1, shuffle=False)

Create the GNN model and train it

In [ ]:
HIDDEN_CHANNLES = 20
MAX_EPOCH = 1000
EPOCH_PATIENCE = 60

model = GNN(data_list[0].num_node_features, HIDDEN_CHANNLES)

model = train_GNN(model, train_data_loader, val_data, MAX_EPOCH, EPOCH_PATIENCE)

Predict labels on test data and compute the L1 and MSE loss

In [ ]:
# Get the model predictions
pred = test_model(model, test_data)

# Compute the associated loss
criterion = torch.nn.L1Loss()
loss = criterion(pred, test_data.y)
print(f"GNN model mean L1 loss: {loss.item():.3f}")

criterion = torch.nn.MSELoss()
loss = criterion(pred, test_data.y)
print(f"GNN model mean MSE loss: {loss.item():.3f}")

Print the predicted labels

In [ ]:
print_predictions(pred, node_ids)